# Neural Style Transfer (VGG19)

This notebook reproduces a classic style-transfer baseline using **VGG19** feature maps and an optimization loop.
It mirrors the parameters referenced in the legacy notes while using modern, readable PyTorch code.

**What you can do**
1. Set content and style images.
2. Choose layer weights and total variation strength.
3. Optimize the output image using LBFGS or Adam.
4. Save the result under `outputs/style_*.png`.


In [ ]:
# !pip install torch torchvision pillow
import torch, torchvision as tv
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import numpy as np
from pathlib import Path
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', device)

def load_image(path, size=512):
    img = Image.open(path).convert('RGB')
    img = tv.transforms.Resize(size)(img)
    img = tv.transforms.ToTensor()(img).unsqueeze(0)
    return img.to(device)

def tensor_to_pil(t):
    t = t.detach().clamp(0,1).cpu().squeeze(0)
    return tv.transforms.ToPILImage()(t)

outdir = Path('outputs'); outdir.mkdir(exist_ok=True)


## Config
Set your content and style images. For a quick test, point both to the same image.


In [ ]:
CONTENT = 'path/to/content.jpg'   # <- change
STYLE   = 'path/to/style.jpg'     # <- change
SIZE    = 512
STEPS   = 300
TV_WEIGHT = 1e-5
STYLE_WEIGHT = 1e4
CONTENT_WEIGHT = 1.0
LR = 0.03
USE_LBFGS = False  # LBFGS often converges faster but uses more memory


## Model and loss setup
We use VGG19 features; style loss is Gram matrix MSE across chosen layers, content loss is MSE at a mid-level conv layer.


In [ ]:
vgg = tv.models.vgg19(weights=tv.models.VGG19_Weights.DEFAULT).features.to(device).eval()
for p in vgg.parameters(): p.requires_grad_(False)

content_layers = ['21']  # relu4_2
style_layers = ['0','5','10','19','28']  # relu1_1 ... relu5_1

def gram_matrix(x):
    b, c, h, w = x.size()
    f = x.view(b*c, h*w)
    G = f @ f.t() / (c*h*w)
    return G

def get_features(x):
    feats = {}
    h = x
    for i, layer in enumerate(vgg):
        h = layer(h)
        if str(i) in set(content_layers + style_layers):
            feats[str(i)] = h
    return feats


In [ ]:
content = load_image(CONTENT, SIZE)
style = load_image(STYLE, SIZE)

target = content.clone().requires_grad_(True)
content_targets = get_features(content)
style_targets = {k: gram_matrix(v) for k,v in get_features(style).items() if k in style_layers}

opt = optim.LBFGS([target]) if USE_LBFGS else optim.Adam([target], lr=LR)

def total_variation(x):
    return ((x[:,:,:-1,:]-x[:,:,1:,:]).abs().mean() + (x[:,:,:,:-1]-x[:,:,:,1:]).abs().mean())


## Optimize
This runs for ~300 steps by default. Adjust `STEPS` or switch to LBFGS to speed up.


In [ ]:
step = 0
t0 = time.time()
while step < STEPS:
    def closure():
        opt.zero_grad()
        feats = get_features(target)
        # Content loss
        c_loss = 0.0
        for l in content_layers:
            c_loss = c_loss + (feats[l] - content_targets[l]).pow(2).mean()
        # Style loss
        s_loss = 0.0
        for l in style_layers:
            s_loss = s_loss + (gram_matrix(feats[l]) - style_targets[l]).pow(2).mean()
        tv_loss = total_variation(target) * TV_WEIGHT
        loss = CONTENT_WEIGHT*c_loss + STYLE_WEIGHT*s_loss + tv_loss
        loss.backward()
        return loss
    if isinstance(opt, optim.LBFGS):
        loss = opt.step(closure)
    else:
        loss = closure()
        opt.step()
    step += 1
    if step % 50 == 0:
        print(f"step {step}/{STEPS} loss={loss.item():.4f}")

elapsed = time.time() - t0
print(f"done in {elapsed:.1f}s")
out_path = outdir / f"style_{int(time.time())}.png"
tensor_to_pil(target).save(out_path)
print("saved:", out_path)


## Notes
- To reproduce a demo quickly, keep size at 512 and switch to LBFGS (`USE_LBFGS=True`).
- Adjust `STYLE_WEIGHT` up for stronger stylization; `TV_WEIGHT` reduces noise and keeps edges clean.
